In [17]:
# default_exp core.library
# export
from ascrum.core.config import config
from pathlib import Path
import yaml
import os
import time
import getpass
import shutil
import time
import mimetypes

In [82]:
# export
class Library:
    def __init__(self, path: Path):
        self.base = path
        self.archive = config.archive_path if config.archive_path else self.base/'.archive'
        self.archive.mkdir(parents=True, exist_ok=True)

    def set_document(self, target: str, source_file=None, source_path=None, create_file=False,
                        mimetype=None, mode='wb', user=None):
        'Move a file into the library'

        mimetype = mimetype if mimetype else mimetypes.guess_type(target)
        dest = self.get_path(target)
        dest.parent.mkdir(parents=True, exist_ok=True)
        self._version(target)

        if source_path:
            shutil.move(source_path, dest)
        elif source_file:
            with dest.open(mode) as w:
                shutil.copyfileobj(source_file, w)
        elif empty_file:
            return dest.open(mode)

    def get_document(self, path, mode='rb'):
        return self.get_path(path).open(mode)

    def mkdir(self, dir=''):
        (self.path/dir).mkdirs(parents=True, exist_ok=True)

    def get_path(self, path):
        return (self.base/path).resolve()

    def _version(self, path):
        if config.versioning:
            p = self.get_path(path)
            if p.exists():
                ts = time.time()
                target = self.archive/f'{path}.{ts}'
                shutil.move(p.as_posix(), target.as_posix())    

    def _get_meta(self,path):
        metafile = self.base/path/'.meta'
        if metafile.exists():
            with p.open() as f:
                return yaml.load(f, Loader=yaml.FullLoader)
        else:
            return {}
    
    def _set_meta(self, path):
        metafile = self.base/path/'.meta'
        with p.open() as f:
            return yaml.load(f, Loader=yaml.FullLoader)

    def list(self, path='.', pattern='*'):
        def list_item(f, meta):
            m = meta.get(f.name, {})
            return {
                'name': f.name, 
                'dir': f.is_dir(), 
                'mime': m.get('mime', None),
                'creator': m.get('creator', None),
            }
        meta = self._get_meta(path)
        p = self.base/path
        return [list_item(f, meta) for f in sorted(p.glob(pattern), 
            key=os.path.getmtime, reverse=True) if not f.name.startswith('.')]


In [83]:
l = Library(Path('/tmp/scrum/library'))


In [84]:
import io
f = io.BytesIO(b"some initial binary data: \x00\x01")
l.set_document('sub/test.txt', source_file=f)


In [85]:
l.list()

[{'name': 'sub', 'dir': True, 'mime': None, 'creator': None}]

In [56]:
[ f for f in Path('/tmp/scrum/library').iterdir()]

[PosixPath('/tmp/scrum/library/sub'), PosixPath('/tmp/scrum/library/.archive')]